# CONFIGURAÇÕES

»» Recursos de interesse:

- Markdown: https://www.markdownguide.org/


## Ativar as bibliotecas de interesse

In [44]:
import os
import sys

In [45]:
import requests
import json

import pickle

import math 
import pandas as pd

## Diretorias


In [46]:
os.getcwd()

'c:\\Users\\gilra\\Desktop\\1semestre'

In [47]:
# When this jupyter notebook is opened, the current working directory is the directory where the notebook is saved.
# However, for a better organization of code, we recomend to change the working directory to the root directory of the project.
# Here a windows terminal command to change the working directory is executed. 
# Please, consider to use the right command if you are using another operative system (UNIX).
# Note that the root path is located one level on the top of the folders tree. Thus, we use the relative path symbol "../" to go up one level.

%cd ../

c:\Users\gilra\Desktop


C:\Users\gilra\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Adicionar os módulos auxiliares

A pasta scripts contém um conjunto de módulos (ficheiros) auxiliares que englobam várias ferramentas (funções) de interesse ao longo do projeto.

In [48]:
# This block of code is used to add the path to the auxiliar modules to the system path.
# Only when the path is added to the system path, the modules can be imported in the notebook as with other
# libraries such as pandas or numpy.

aux_modules_path = os.path.abspath(os.path.join('./scripts'))
if aux_modules_path not in sys.path:
    sys.path.append(aux_modules_path)

### Exemplo de uso dos módulos auxiliares


In [49]:
from scripts.auxiliar_tools import *

print(print_hello_world())

Hello World!


## Configuração Chave API

In [50]:
from decouple import config

# https://medium.com/@alensabu12xtz/configure-environment-variables-with-python-decouple-b5f5446381a3

MY_SCOPUS_API_KEY = config('MY_SCOPUS_API_KEY')

print('my scopus api key is :', MY_SCOPUS_API_KEY) 

my scopus api key is : dc7c18eddcd470e03ed6f72cf4a11585


# RECOLHA DE DADOS - Parte 1
Pesquisa e recolha de resultados com a API Scoupus API

## Análise exploratória dos resultados de pesquisa

### Exploratório: 1. Pesquisa API

In [51]:
# IMPORTANTE: https://dev.elsevier.com/sc_search_tips.html

user_query = '"impact" AND "high* education" AND "artificial intelligence"'

response = requests.get("https://api.elsevier.com/content/search/scopus",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_SCOPUS_API_KEY},
                         
                    params={    'query' : user_query,
                                #'start' : "1"
                        }

                        )

print(response.url)



https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model


### Exploratório: 2. Analisar resultados / resposta API

In [52]:
print(response.status_code)

200


In [53]:
# A simples invocação da resposta da API (em formato json) imprime o contéudo obtido com a respetiva syntaxe json
# No entanto, como facilmente se depreende desta forma é dificil discernir a estrutura hierárquica dos dados e, desta forma, extrair os diferentes dados em cada nível
# por forma a transformar os dados numa estrutura de dados mais amigável (por exemplo, um data.frame da livraria "pandas", que nos é uma estrutura de dados mais familiar
# para manipulação)
results = response.json()
results


{'search-results': {'opensearch:totalResults': '403',
  'opensearch:startIndex': '0',
  'opensearch:itemsPerPage': '25',
  'opensearch:Query': {'@role': 'request',
   '@searchTerms': 'data mining housing automated valuation model',
   '@startPage': '0'},
  'link': [{'@_fa': 'true',
    '@ref': 'self',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=data+mining+housing+automated+valuation+model',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'first',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=data+mining+housing+automated+valuation+model',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'next',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=25&count=25&query=data+mining+housing+automated+valuation+model',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'last',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=378&count=

In [54]:
# Podemos assim invocar a função "jprint" que definimos inicialmente para obtermos uma visualização mais amigável da estrutura de dados codificada no json
# Esta visualização permite-nos ainda identificar a "localização" (na hierarquia) dos dados de interesse e, portanto, as respetivas chaves (que nos permitem
# aceder a esses mesmos dados)
jprint(response.json())

'{\n      "search-results": {\n            "entry": [\n                  {\n                        "@_fa": "true",\n                        "affiliation": [\n                              {\n                                    "@_fa": "true",\n                                    "affiliation-city": "Al Ain",\n                                    "affiliation-country": "United Arab Emirates",\n                                    "affilname": "College of Business and Economics"\n                              }\n                        ],\n                        "article-number": "3068",\n                        "citedby-count": "0",\n                        "dc:creator": "Al Abdulsalam A.S.",\n                        "dc:identifier": "SCOPUS_ID:85207330426",\n                        "dc:title": "A Novel Data Analytics Methodology for Analyzing Real Estate Brokerage Markets with Case Study of Dubai",\n                        "eid": "2-s2.0-85207330426",\n                        "link": [

In [55]:
results["search-results"]['entry'][0]["affiliation"]


[{'@_fa': 'true',
  'affilname': 'College of Business and Economics',
  'affiliation-city': 'Al Ain',
  'affiliation-country': 'United Arab Emirates'}]

In [56]:
'affiliation' in results["search-results"]['entry'][0]

True

### Exploratório: 3. Número total de resultados obtidos na pesquisa

In [57]:
number_of_articles_retrieved = results["search-results"]["opensearch:totalResults"]
number_of_articles_retrieved

'403'

In [58]:
number_of_articles_perResultPage = results["search-results"]["opensearch:itemsPerPage"]
number_of_articles_perResultPage

'25'

### Exploratório: 4. Obter dados de cada um dos resultados de pesquisa e transformar numa estrutura de dados amigável (pandas -» data.frame )

#### Opção 1

In [59]:
df = pd.DataFrame.from_records(results["search-results"]['entry'] )
df[:3]

,@_fa,link,prism:url,dc:identifier,eid,dc:title,dc:creator,prism:publicationName,prism:eIssn,prism:volume,...,subtypeDescription,article-number,source-id,openaccess,openaccessFlag,prism:issn,freetoread,freetoreadLabel,pii,prism:isbn
0,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85207330426,2-s2.0-85207330426,A Novel Data Analytics Methodology for Analyzi...,Al Abdulsalam A.S.,Buildings,20755309,14,...,Article,3068,26980,1,True,NaN,NaN,NaN,NaN,NaN
1,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85153357457,2-s2.0-85153357457,An innovative methodology for the assessment o...,Tajani F.,Smart and Sustainable Built Environment,20466102,13,...,Article,NaN,21100385605,0,False,20466099,"{'value': [{'$': 'all'}, {'$': 'repository'}, ...","{'value': [{'$': 'All Open Access'}, {'$': 'Gr...",NaN,NaN
2,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85205255315,2-s2.0-85205255315,Stacked Ensemble Model for the Automatic Valua...,Kim W.,Land,2073445X,13,...,Article,1436,21100811521,1,True,NaN,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN


#### Opção 2

In [60]:
df = pd.json_normalize(results["search-results"],['entry'] )
df[:3]

,@_fa,link,prism:url,dc:identifier,eid,dc:title,dc:creator,prism:publicationName,prism:eIssn,prism:volume,...,subtypeDescription,article-number,source-id,openaccess,openaccessFlag,prism:issn,freetoread.value,freetoreadLabel.value,pii,prism:isbn
0,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85207330426,2-s2.0-85207330426,A Novel Data Analytics Methodology for Analyzi...,Al Abdulsalam A.S.,Buildings,20755309,14,...,Article,3068,26980,1,True,NaN,NaN,NaN,NaN,NaN
1,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85153357457,2-s2.0-85153357457,An innovative methodology for the assessment o...,Tajani F.,Smart and Sustainable Built Environment,20466102,13,...,Article,NaN,21100385605,0,False,20466099,"[{'$': 'all'}, {'$': 'repository'}, {'$': 'rep...","[{'$': 'All Open Access'}, {'$': 'Green'}]",NaN,NaN
2,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85205255315,2-s2.0-85205255315,Stacked Ensemble Model for the Automatic Valua...,Kim W.,Land,2073445X,13,...,Article,1436,21100811521,1,True,NaN,"[{'$': 'all'}, {'$': 'publisherfullgold'}]","[{'$': 'All Open Access'}, {'$': 'Gold'}]",NaN,NaN


In [61]:
df.columns

Index(['@_fa', 'link', 'prism:url', 'dc:identifier', 'eid', 'dc:title',
       'dc:creator', 'prism:publicationName', 'prism:eIssn', 'prism:volume',
       'prism:issueIdentifier', 'prism:pageRange', 'prism:coverDate',
       'prism:coverDisplayDate', 'prism:doi', 'citedby-count', 'affiliation',
       'prism:aggregationType', 'subtype', 'subtypeDescription',
       'article-number', 'source-id', 'openaccess', 'openaccessFlag',
       'prism:issn', 'freetoread.value', 'freetoreadLabel.value', 'pii',
       'prism:isbn'],
      dtype='object')

### Exploratório: 5. Aceder a dados em diferentes níveis hierárquicos

In [62]:
df_affiliation = pd.json_normalize(results["search-results"],['entry', 'affiliation'] )
df_affiliation.columns
#df['prism:doi']
#df_affiliation



Index(['@_fa', 'affilname', 'affiliation-city', 'affiliation-country'], dtype='object')

In [63]:
df_affiliation[:5]



,@_fa,affilname,affiliation-city,affiliation-country
0,true,College of Business and Economics,Al Ain,United Arab Emirates
1,true,Sapienza Università di Roma,Rome,Italy
2,true,Konkuk Business School,Seoul,South Korea
3,true,Universiti Teknologi PETRONAS,Seri Iskandar,Malaysia
4,true,The University of Waikato,Hamilton,New Zealand


### Exploratório: 5.1 Aceder a dados em diferentes níveis hierárquicos »» POSSÍVEIS DIFICULDADES

Por vezes, nos resultados de pesquisa alguns items podem não conter certos elementos / informação codificada em níveis hierárquicos
inferiores do json.
Por exemplo, a informação sobre a afiliação dos autores pode estar omissa. 
Nestes casos, ao usar a função "pd.json_normalize" para navegar para níveis inferiores pode devolver um erro e inviabilizar o processo automatizado (isto porque a função
"json_normalize" não permite uma forma expedita de solucionar este problema).

Uma estratégia "naife" para ultrapassar esta questão passa por  transformar os dados para um data.frame considerando
apenas o nível inicial da hierarquia do json (ou seja, utilizar a função json_normalize como fizemos para obter o objeto "df" anteriormente).
Como verificamos, certos elementos do data.frame "df" armazenam dados em formato json / dicionário nas suas células.
Ao invés de utilizarmos a função "json_normalize" para obtermos dados nos níveis hierárquicos inferiores, podemos desenvolver o nosso próprio algoritmo
por forma a solucionarmos os problemas de dados omissos.

Assim, uma implementação alternativa do ponto 5, para os dados da coluna "affiliation" seria algo como o que se ilustra no código seguinte.

NOTA: reparem que se optarem por esta estratégia deverão alterar o código da seccção "Automatização global" em conformidade!

In [64]:
import numpy as np 

i=0 #devem utilizar um índice (de linha) para o qual existam dados na coluna "affiliation" do vosso data.frame «df»
AFF_COL_NAMES = pd.json_normalize(df.affiliation[0]).columns

df_affiliation_v2 = pd.DataFrame(columns=AFF_COL_NAMES)

for i in range(len(df.affiliation)) :
    if  type(df.affiliation.iloc[i]) == list :
        df_aux = pd.json_normalize(df.affiliation.iloc[i])
        df_affiliation_v2 = pd.concat([df_affiliation_v2, df_aux], ignore_index=True)

    else:
        emptydf = pd.DataFrame(np.nan, index=[0],columns=AFF_COL_NAMES) 
        df_affiliation_v2 = pd.concat([df_affiliation_v2,emptydf], ignore_index=True)

In [65]:
df_affiliation_v2[:5]

,@_fa,affilname,affiliation-city,affiliation-country
0,true,College of Business and Economics,Al Ain,United Arab Emirates
1,true,Sapienza Università di Roma,Rome,Italy
2,true,Konkuk Business School,Seoul,South Korea
3,true,Universiti Teknologi PETRONAS,Seri Iskandar,Malaysia
4,true,The University of Waikato,Hamilton,New Zealand


### Exploratório: 6. Combinar dados de diferentes hierarquias num único "data.frame"
https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html 

In [66]:
FIELDS = ['dc:title', "prism:doi", 'dc:creator', 'citedby-count', 'openaccess']
df[FIELDS][:3]

,dc:title,prism:doi,dc:creator,citedby-count,openaccess
0,A Novel Data Analytics Methodology for Analyzi...,10.3390/buildings14103068,Al Abdulsalam A.S.,0,1
1,An innovative methodology for the assessment o...,10.1108/SASBE-12-2022-0274,Tajani F.,2,0
2,Stacked Ensemble Model for the Automatic Valua...,10.3390/land13091436,Kim W.,0,1


In [67]:
df_all = pd.merge(df[FIELDS], df_affiliation_v2, how='left', left_index=True, right_index=True)
df_all[:3]


,dc:title,prism:doi,dc:creator,citedby-count,openaccess,@_fa,affilname,affiliation-city,affiliation-country
0,A Novel Data Analytics Methodology for Analyzi...,10.3390/buildings14103068,Al Abdulsalam A.S.,0,1,true,College of Business and Economics,Al Ain,United Arab Emirates
1,An innovative methodology for the assessment o...,10.1108/SASBE-12-2022-0274,Tajani F.,2,0,true,Sapienza Università di Roma,Rome,Italy
2,Stacked Ensemble Model for the Automatic Valua...,10.3390/land13091436,Kim W.,0,1,true,Konkuk Business School,Seoul,South Korea


## Automatização global
Recolha de resultados individuais da API, seleção de dados de interesse, criação de estrutura de dados amigável

In [68]:
user_query = "data mining housing automated valuation model"
LEVEL0_FIELDS_OF_INTEREST = ['dc:title', "prism:doi", 'dc:creator']
AFFILIATION_FIELDS_OF_INTEREST = ['affilname', 'affiliation-city', 'affiliation-country']
results_aux = ''

df_results_list = pd.DataFrame(columns=LEVEL0_FIELDS_OF_INTEREST+AFFILIATION_FIELDS_OF_INTEREST)

cursor = "*"

for i in range(0,math.ceil(int(number_of_articles_retrieved)/int(number_of_articles_perResultPage))) :
#for i in range(0,1) :
    
    
    response = requests.get("https://api.elsevier.com/content/search/scopus",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_SCOPUS_API_KEY},
                         
                    params={    'query' : user_query,
                                'cursor' : cursor
                        }

                        )
    print(i)
    print(response.url)
    print(response.status_code)

    if(response.status_code == 200):
        results_aux = response.json()

        df_level1_aux = pd.DataFrame.from_records(results_aux["search-results"]['entry'] )
        df_level1_aux = df_level1_aux[LEVEL0_FIELDS_OF_INTEREST]
                
        df_affil_aux = pd.json_normalize(results["search-results"],['entry', 'affiliation'] )
        df_affil_aux = df_affil_aux[AFFILIATION_FIELDS_OF_INTEREST]
                
        df_all_aux = pd.merge(df_level1_aux, df_affil_aux, how='left', left_index=True, right_index=True)
        
        df_results_list = df_results_list.append(df_all_aux, ignore_index=True)


        cursor = results_aux["search-results"]["cursor"]['@next']
    

0
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
1
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
2
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
3
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
4
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
5
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
6
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
7
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
8
https://api.elsevier.com/content/search/scopus?query=data+mining+housi

In [69]:
user_query = "data mining housing automated valuation model"
LEVEL0_FIELDS_OF_INTEREST = ['dc:title', "prism:doi", 'dc:creator']
AFFILIATION_FIELDS_OF_INTEREST = ['affilname', 'affiliation-city', 'affiliation-country']

df_results_list = pd.DataFrame(columns=LEVEL0_FIELDS_OF_INTEREST+AFFILIATION_FIELDS_OF_INTEREST)

cursor = "*"

for i in range(0,math.ceil(int(number_of_articles_retrieved)/int(number_of_articles_perResultPage))) :
#for i in range(0,1) :
    
    
    response = requests.get("https://api.elsevier.com/content/search/scopus",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_SCOPUS_API_KEY},
                         
                    params={    'query' : user_query,
                                'cursor' : cursor
                        }

                        )
    print(i)
    print(response.url)
    print(response.status_code)

    if(response.status_code == 200):
        results_aux = response.json()

        df_level1_aux = pd.DataFrame.from_records(results_aux["search-results"]['entry'] )
        df_level1_aux = df_level1_aux[LEVEL0_FIELDS_OF_INTEREST]
        
        df_affil_aux_out =pd.DataFrame(columns=AFF_COL_NAMES)

        for i in range(len(results_aux["search-results"]['entry'])) :

            if 'affiliation' in results["search-results"]['entry'][i]:
                
                df_affil_aux_in = pd.json_normalize(results["search-results"]['entry'][i]['affiliation'])
                df_affil_aux_in = df_affil_aux_in[AFFILIATION_FIELDS_OF_INTEREST]
                df_affil_aux_out = pd.concat([df_affil_aux_in, df_affil_aux_out], axis=0, ignore_index=True)
               

            else:

                df_affil_aux_in = pd.DataFrame(np.nan, index=[0],columns=AFF_COL_NAMES)
                
        
        df_all_aux = pd.merge(df_level1_aux, df_affil_aux_out, how='left', left_index=True, right_index=True)
        df_results_list = pd.concat([df_results_list, df_all_aux], ignore_index=True)
        cursor = results_aux["search-results"]["cursor"]['@next']
        




        
    

0
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
1
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
2
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
3
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
4
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
5
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
6
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
7
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&cursor=%2A
403
8
https://api.elsevier.com/content/search/scopus?query=data+mining+housi

In [70]:
len(results_aux["search-results"]['entry'])

TypeError: string indices must be integers, not 'str'

In [ ]:
results["search-results"]['entry'][20]['affiliation']

In [ ]:
pd.json_normalize(results["search-results"]['entry'][20]['affiliation'])

In [ ]:
"affiliation" in results["search-results"]['entry'][20]

In [ ]:
jprint(response.json())

In [ ]:
df_results_list[:200]

In [ ]:
df_results_list.to_csv('../data/interim/df_results_list_records.csv', index=False)

# RECOLHA DE DADOS - Parte 2
Recolha do contéudo dos artigos selecionados com a API Scopus

NOTA: Como facilmente se verifica na [página](https://dev.elsevier.com/api_docs.html) os artigos completos 
estão disponíveis no serviço «ScienceDirect APIs», especificamente na API «Article Retrieval» (documentação [aqui](https://dev.elsevier.com/documentation/ArticleRetrievalAPI.wadl) )

## Recolha de artigos completos
Utilizando o identificador DOI

### Análise exploratória

In [ ]:
dois = df_results_list['prism:doi']
dois[:3]

In [ ]:
response_article = requests.get("https://api.elsevier.com/content/article/doi/"+dois[2],
                    headers={ 'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY},
                         
                    params={   }

                        )
print(response_article.url)
print(response_article.status_code)

In [ ]:
#jprint(response_article.json())

In [ ]:
type(dois)

### Implementação global

In [ ]:
# - a definir -


## Recolha de resumos (dos artigos) 

Recolha do contéudo dos resumos dos artigos selecionados com a API Scopus (usando o identificador DOI guardado anteriormente)

NOTA: Como facilmente se verifica na [página](https://dev.elsevier.com/api_docs.html) os resumos podem ser obtidos de duas formas:
* Através do serviço «ScienceDirect APIs» e especificamente da API «Article Retrieval» (documentação [aqui](https://dev.elsevier.com/documentation/ArticleRetrievalAPI.wadl) ) descrita anteriormente
* Através do serviço «Scopus APIs» (usado anteriormente para fazer a pesquisa) e especificamente da API «Abstract Retrieval» (documentação [aqui](https://dev.elsevier.com/documentation/AbstractRetrievalAPI.wadl))

### Análise Exploratória

In [ ]:
dois = df_results_list['prism:doi']

In [ ]:
response_abst = requests.get("https://api.elsevier.com/content/abstract/doi/"+dois[114],
                    headers={#'Accept': 'application/json',
                    'Accept': 'application/json',
                    'X-ELS-APIKey': MY_API_KEY},
                    
                    params={   }
                )
print(response_abst.url)
print(response_abst.status_code)   

In [ ]:
results_abstr = response_abst.json()
results_abstr

In [ ]:
jprint(response_abst.json())

In [ ]:
resumo = results_abstr["abstracts-retrieval-response"][ "coredata"]['dc:description']
resumo

### Implementação Global

In [ ]:
dois = df_results_list['prism:doi']
dois

In [ ]:
abstract_list = []

i=0
for x in dois :
    print(x)
    response_abst = requests.get("https://api.elsevier.com/content/abstract/doi/"+str(x),
                    headers={
                    'Accept': 'application/json',
                    'X-ELS-APIKey': MY_API_KEY},
                    
                    params={   }
                )
    
    print(response_abst.url)
    print(response_abst.status_code)
    print(i)
    i=i+1

    if(response_abst.status_code == 200):
        results_abst = response_abst.json()
        if 'dc:description' in results_abst["abstracts-retrieval-response"][ "coredata"]:
            abstract_list.append( results_abst["abstracts-retrieval-response"][ "coredata"]['dc:description'] )
        else:
            abstract_list.append("NULL")
    else:
        abstract_list.append("NULL")




In [ ]:
abstract_list[0]

In [ ]:
type(abstract_list[0])

# GUARDAR DADOS RECOLHIDOS E PROCESSADOS

In [ ]:
df_results_list.to_csv('../data/input/'+'search_results.csv',  index=False, encoding = "utf-8")

pd.DataFrame(abstract_list,  columns =['Abstract']).to_csv('../data/input/'+'search_results_abstracts.csv', index=False, encoding = "utf-8")

In [ ]:
# Save the file
#pickle.dump( df_results_list, file = open(path_dados+"ICD2122_TextMining_DataStructures.pickle", "wb"))

# Reload the file
#test_grouped_df_reloaded = pickle.load(open(path_dados+"ICD2122_TextMining_DataStructures.pickle", "rb"))